# Understanding Table Formats

Now we have a good grasp of what file formats we can use to store data in, we can turn our attention to table formats.

The fundamental question is, how do we define a table? We could define it as a single file, but that would be problematic as we add more data. Most likely, we would want to define a number of files as a table. Thus a common convention is that a table is a directory of files. Next, we need to store the relationship between a table name and the physical directories. 

In addition to the physical location of the data, we can store the schema of the datafiles as well to avoid having to open files to get the schema.


## Apache Hive

![Apache Hive Logo](images/logos/apache_hive_logo.png)

The first implementation of these ideas was Apache Hive. Instead of having to write Java-based MapReduce code, Hive let the enduser write SQL, which would then be compiled to MapReduce jobs on top of HDFS. This let endusers write HiveQL, a SQL-alike language, but keep the benefits of having data in Hadoop HDFS. 

Hive keeps the metadata of tables -> locations as well as schemas, bucketing information, serde information as well as user-defined metadata. Since Hive was a key component of many enterprise data lakes, many modern tools have the ability to 

One key concept that Hive introduced was `partitioning`. As discussed previously, the key optimization when working with files, is to avoid having to read them. One approach is to have the directories themselves be a key-value pair, allowing the query engine to skip reading files in a directory if the key-value pair doesn't match the predicate.

![Hive Partitioning](images/hive_partitioning.png)

Given that we know our query patterns, this partitioning pattern lets us skip all files not related to a given year/month. 

The downside is that the person writing the query needs to know the partitioning structure of the data to take advantage of this. 

We can't write `SELECT * FROM TABLE WHERE created_date = '2024-01-01` as there is no partition `created_data=2024-01-01`. We would need to write `SELECT * FROM TABLE WHERE year = '2024' and month = '01' and day = '01'`. Try to think how to fetch all records after that date...



# Apache Iceberg

![Apache Iceberg Logo](images/logos/Apache_Iceberg_Logo.svg)

Apache Iceberg was developed as a better alternative to Apache Hive at Netflix in 2017 and become a top-level Apache project in 2024. 
The Iceberg table format improves upon Hive in a number of ways, the most important being ACID transactions, guaranteeing that readers won't read dirty data. 
Apache Iceberg also introduces a number of other benefits, including adding full schema evolution, hidden partitioning as well as time travel and rollback features. 


## Apache Iceberg Architecture
![Apache Iceberg Architecture](images/iceberg_architecture.jpeg)
> Source: https://www.dremio.com/resources/guides/apache-iceberg-an-architectural-look-under-the-covers/

One key difference between Hive and Iceberg, is that Iceberg defines tables at the file-level. The metadata files keep track of which files are in scope for a given table, which is the key to many of the performance benefits

## Iceberg Catalogue

Iceberg uses a catalog to keep track of the current metadata version, as well as ensure atomic updates to the metadata pointer. The catalogue can provide a number of other features besides this, but the main job is the management of the current metadata version.

Originally, each catalogue type needed direct integrations in the ecosystem to be supported. So PyIceberg only supported Hive, DynamoDB, SQL and Glue catalogues because each integration had to be programmed in separately. In 2022, the Iceberg project launched a REST specification for Iceberg catalogues that meant catalogues could implement a shared, common standard, easing the burden of integrating. 

Work has been ongoing to implement the REST standard into the various catalogues, but recently adoption has taken off, with Snowflake open-sourcing their [Polaris catalogue](https://www.snowflake.com/blog/introducing-polaris-catalog/), Databricks both [acquiring Tabular](https://www.databricks.com/blog/databricks-tabular) (the company founded by the Iceberg creators) and open-sourcing their [Unity Catalogue](https://www.databricks.com/blog/open-sourcing-unity-catalog), Hive and Nessie announcing REST compatibility, as well as the release of multiple new catalogs based on the spec such as [Gravitino](https://gravitino.apache.org/) or the [Hansetag catalog](https://github.com/hansetag/iceberg-catalog)

To demonstrate the extra features that a catalog can offer, we will be using the [Nessie](https://projectnessie.org/) catalogue

